<a href="https://colab.research.google.com/github/CodingTomo/TE-UrbanSound8K/blob/master/MFCC_extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
! pip install torchaudio

In [5]:
import os
import pandas as pd
import torchaudio
import torch

In [6]:
metadata = pd.read_csv('/content/drive/Shared drives/TE-UrbanSound/UrbanSound8K/metadata/UrbanSound8K.csv')
SAMPLE_RATE = 8000
MELKWARGS = {'n_fft': 512, 'hop_length': 128}
N_MFCC = 40
N_WAVEFORM = metadata.shape[0]

In [7]:
def stereo_to_mono(waveform):
    waveform_mono = torch.mean(waveform, dim=0, keepdim=True)
    return waveform_mono

In [8]:
def resampling_8k(waveform, sr, SAMPLE_RATE):
    waveform = torchaudio.transforms.Resample(new_freq=SAMPLE_RATE, orig_freq=sr)(waveform)
    return waveform


In [10]:
data = {'name': [],
        'mfcc': [],
        'label': []
        }
i=0

for root, dirs, files in os.walk('/content/drive/Shared drives/TE-UrbanSound/UrbanSound8K'):
    for name in files:
        if name.endswith(".wav"):
            complete_path = root + "/" + name
            waveform, sr = torchaudio.load(complete_path)
            if waveform.shape[0] == 2:
                waveform = stereo_to_mono(waveform)
            if sr != SAMPLE_RATE:
                waveform = resampling_8k(waveform, sr, SAMPLE_RATE)
            feature = torchaudio.transforms.MFCC(sample_rate=SAMPLE_RATE,
                                                n_mfcc=N_MFCC,
                                                melkwargs=MELKWARGS)(waveform)
            data['name'].append(name)
            data['mfcc'].append(feature)
            data['label'].append(metadata.loc[metadata['slice_file_name'] == name]['classID'].iloc[0])
            i=i+1
            if i%100==0:
              print('-'*40)
              print('Processed {:.0%}'.format(i/N_WAVEFORM))

----------------------------------------
Processed 1%
----------------------------------------
Processed 2%


KeyboardInterrupt: ignored